# Импорт используемых библиотек

In [9]:
!pip install rdflib
!pip install SPARQLWrapper
!apt-get install netbase

from rdflib import URIRef, BNode, Literal, Namespace, Graph
from rdflib.namespace import RDF, RDFS
from SPARQLWrapper import SPARQLWrapper, JSON

     |████████████████████████████████| 235kB 8.7MB/s 
     |████████████████████████████████| 51kB 6.1MB/s 
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  netbase
0 upgraded, 1 newly installed, 0 to remove and 34 not upgraded.
Need to get 12.7 kB of archives.
After this operation, 45.1 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/main amd64 netbase all 5.4 [12.7 kB]
Fetched 12.7 kB in 0s (36.4 kB/s)
Selecting previously unselected package netbase.
(Reading database ... 160706 files and directories currently installed.)
Preparing to unpack .../archives/netbase_5.4_all.deb ...
Unpacking netbase (5.4) ...
Setting up netbase (5.4) ...


# Classes URI

In [14]:
musicalGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#MusicalGroup')
rockGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#RockGroup')
rapGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#RapGroup')
popGroup = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#PopGroup')

musicalGenre = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#MusicalGenre')
rockMusic = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#RockMusic')
popMusic = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#PopMusic')
hipHopMusic = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#HipHopMusic')

musician = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Musician')
award = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Award')
country = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Country')
language = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#Language')

# Object Properties URI

In [15]:
bornIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#bornIn')
foundedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#foundedIn')
hasDeserved = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#hasDeserved')
hasGenre = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#hasGenre')
hasMember = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#hasMember')
playedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#playedIn')
speaksIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#speaksIn')
rdftype = URIRef("http://www.w3.org/1999/02/22-rdf-syntax-ns#type") 

# Data Properties URI

In [ ]:
careerStarted = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#careerStarted')
careerOver = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#careerOver')
diedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#diedIn')
disbandedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#disbandedIn')
foundedIn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#foundedIn')
wasBorn = URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#wasBorn')

In [ ]:
graph = Graph().parse('MusicBands.owl', format='application/rdf+xml')

# Подключение к WikiData

## добавление музыкальных групп

In [ ]:
def checkIfHasLabel(s):
    if s[0] != "Q":
        return True 
    try: 
        int(s)
        return True
    except ValueError:
        return False

def add_to_graph(prop: str, id: str, uri: URIRef, ):
  sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

  sparql.setQuery("""
  SELECT ?item ?itemLabel 
  WHERE 
  { 
    ?item wdt:""" + prop + """ wd:""" + id + """.
    SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                             ?item rdfs:label ?itemLabel }
  }
  """)
  sparql.setReturnFormat(JSON)
  results = sparql.query().convert()
  i = 0
  for result in results["results"]["bindings"]:
    name = result["itemLabel"]["value"]
    if not checkIfHasLabel(name):
      continue
    name = name.replace(" ", "_").replace('"','').replace('^', '').replace('`', '').replace('>', '').replace('<', '')
    item = URIRef(f'http://www.musicbands.com/ontologies/MusicBands.owl#{ name }')
    graph.add((item, RDF.type, uri))
    i = i + 1
  return i

In [ ]:
#add_to_graph("P31", "Q5741069", rockGroup)
#add_to_graph("P31", "Q19464263", rapGroup)
#add_to_graph("P31", "Q2491498", popGroup)

add_to_graph("P279", "Q11399", rockMusic)
add_to_graph("P279", "Q11401", hipHopMusic)
add_to_graph("P279", "Q37073", popMusic)

add_to_graph("P31", "Q6256", country)
#add_to_graph("P106", "Q639669", musician)
add_to_graph("P31", "Q1288568", language)
add_to_graph("P31", "Q1364556", award)

HTTPError: ignored

In [ ]:
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")

sparql.setQuery("""
SELECT ?item ?itemLabel 
WHERE 
{ 
  ?item wdt:""" + prop + """ wd:""" + id + """.
  SERVICE wikibase:label { bd:serviceParam wikibase:language "[AUTO_LANGUAGE],en".
                           ?item rdfs:label ?itemLabel }
}
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()
i = 0
for result in results["results"]["bindings"]:
  name = result["itemLabel"]["value"]
  if not checkIfHasLabel(name):
    continue
  name = name.replace(" ", "_").replace('"','').replace('^', '').replace('`', '').replace('>', '').replace('<', '')
  item = URIRef(f'http://www.musicbands.com/ontologies/MusicBands.owl#{ name }')
  graph.add((item, RDF.type, uri))
  i = i + 1

Сохранить в файл "MusicBands_populated.rdf"

In [89]:
graph.serialize(destination='/content/drive/MyDrive/colab_data/MusicBands_populated_simple3.rdf')

In [60]:
graph = Graph().parse('/content/drive/MyDrive/colab_data/MusicBands_populated_simple.rdf', format='application/rdf+xml')

In [88]:
qres = graph.query(
    """
    PREFIX g: <http://www.musicbands.com/ontologies/MusicBands.owl#>
    SELECT ?a ?b
    WHERE {
        ?a ?b <http://www.musicbands.com/ontologies/MusicBands.owl#The_Good_Life> .
        
    }
       """)
for row in qres:
    graph.remove((row[0], row[1], URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#The_Good_Life')))
    print(row)

(rdflib.term.URIRef("http://www.musicbands.com/ontologies/MusicBands.owl#The_'89_Cubs"), rdflib.term.URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#hasMember'))


In [80]:
qres = graph.query(
    """
    PREFIX g: <http://www.musicbands.com/ontologies/MusicBands.owl#>
    SELECT ?a ?b ?c
    WHERE {
        ?a rdf:type <http://www.musicbands.com/ontologies/MusicBands.owl#MusicalGroup> .
        ?a ?b ?c.
        FILTER (?b = <http://www.musicbands.com/ontologies/MusicBands.owl#playedIn>) .
    }
       """)
for row in qres:
    print(row)

SPARQL запросы

1 - какие жанры были популярны в 1970-х годах?

In [ ]:
qres = graph.query(
    """
    PREFIX g: <http://www.musicbands.com/ontologies/MusicBands.owl#>
    SELECT ?genre (COUNT(?group) AS ?count)
    WHERE {
        ?group g:foundedIn ?data .
        ?group g:hasGenre ?genre .
        FILTER (?data > 1969 && ?data < 1980) . 

    }
    GROUP BY ?genre
    ORDER BY DESC(?count) LIMIT 5
       """)
for row in qres:
    print("%s -> %s" % row)


http://www.musicbands.com/ontologies/MusicBands.owl#rock_music -> 42
http://www.musicbands.com/ontologies/MusicBands.owl#progressive_rock -> 36
http://www.musicbands.com/ontologies/MusicBands.owl#hard_rock -> 29
http://www.musicbands.com/ontologies/MusicBands.owl#punk_rock -> 20
http://www.musicbands.com/ontologies/MusicBands.owl#new_wave -> 18


2 - в каком году появилась первая progressive rock группа?


In [ ]:
qres = graph.query(
    """
    PREFIX g: <http://www.musicbands.com/ontologies/MusicBands.owl#>
    SELECT ?group ?date
    WHERE {
        ?group g:hasGenre <http://www.musicbands.com/ontologies/MusicBands.owl#progressive_rock> .
        ?group g:foundedIn ?date
    }
    ORDER BY ?date LIMIT 1
       """)
for row in qres:
    print("%s -> %s" % row)

http://www.musicbands.com/ontologies/MusicBands.owl#The_Moody_Blues -> 1964


3 - в каком году в США появилось больше всего музыкальных групп?

In [ ]:
qres = graph.query(
    """
    PREFIX g: <http://www.musicbands.com/ontologies/MusicBands.owl#>
    SELECT ?date (COUNT(?group) AS ?count)
    WHERE {
        ?group g:countryOfOrigin <http://www.musicbands.com/ontologies/MusicBands.owl#United_States_of_America> .
        ?group g:foundedIn ?date
    }
    GROUP BY ?date
    ORDER BY DESC(?count) LIMIT 5
       """)
for row in qres:
    print("%s -> %s" % row)

2005 -> 406
2004 -> 365
2006 -> 356
2003 -> 348
2001 -> 340


4 - какой жанр был наиболее популярен в Англии в 90-е?

In [ ]:
qres = graph.query(
    """
    PREFIX g: <http://www.musicbands.com/ontologies/MusicBands.owl#>
    SELECT ?genre
    WHERE {
        ?group g:countryOfOrigin <http://www.musicbands.com/ontologies/MusicBands.owl#United_Kingdom> .
        ?group g:hasGenre ?genre .
        ?group g:foundedIn ?date .
        FILTER (?date > 1989 && ?date < 2000) . 

    }
    GROUP BY ?genre
    ORDER BY DESC(?count) LIMIT 5
       """)
for row in qres:
    print(row)

(rdflib.term.URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#pop_rock'),)
(rdflib.term.URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#post-Britpop'),)
(rdflib.term.URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#art_rock'),)
(rdflib.term.URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#pop_music'),)
(rdflib.term.URIRef('http://www.musicbands.com/ontologies/MusicBands.owl#alternative_rock'),)


# Reasoning

In [2]:
! pip install Owlready2
from owlready2 import *

     |████████████████████████████████| 23.7MB 1.8MB/s 
  Created wheel for Owlready2: filename=Owlready2-0.31-cp37-cp37m-linux_x86_64.whl size=20409729 sha256=a751080719f980bbe848b09ef591dc9cf607a7fa4c19f1c960a41772f5cafc9c
  Stored in directory: /root/.cache/pip/wheels/9d/17/e4/54fb8381c51746a0d686b06ef3e3f607789defd8e1580bcb1b
Successfully built Owlready2


In [ ]:
import os       #importing os to set environment variable
def install_java():
  !apt-get install -y openjdk-8-jdk-headless -qq > /dev/null      #install openjdk
  os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"     #set environment variable
  !java -version       #check java version
install_java()


openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [90]:
from owlready2 import *
import owlready2

onto = get_ontology("/content/drive/MyDrive/colab_data/MusicBands_populated_simple3.rdf").load()
sync_reasoner()

onto_inferred = get_ontology("http://inferrences/")
onto_inferred.save("kek.nt", format="ntriples") 

* Owlready2 * Running HermiT...
    java -Xmx2000M -cp /usr/local/lib/python3.7/dist-packages/owlready2/hermit:/usr/local/lib/python3.7/dist-packages/owlready2/hermit/HermiT.jar org.semanticweb.HermiT.cli.CommandLine -c -O -D -I file:////tmp/tmp88eggf66


OwlReadyInconsistentOntologyError: ignored

In [6]:
list(onto.

[MusicBands_populated_simple.bornIn,
 MusicBands_populated_simple.countryOfOrigin,
 MusicBands_populated_simple.hasDeserved,
 MusicBands_populated_simple.hasGenre,
 MusicBands_populated_simple.hasMember,
 MusicBands_populated_simple.playedIn,
 MusicBands_populated_simple.speaksIn]

In [ ]:
with onto:
     class Group(onto.MusicalGroup):
         def take(self): print("group")

     class Genre(onto.MusicalGenre):
         pass

     class has_genre(Group >> Genre):
         python_name = "genres"

     class SingleGenreGroup(Group):
         equivalent_to = [Group & has_genre.exactly(1, Genre)]
         def take(self): print("multi genre group")

     class MultiGenreGroup(Group):
         equivalent_to = [Drug & has_genre.exactly(1, Genre)]
         def take(self): print("single genre group")

In [ ]:
class SingleGenreGroup(onto.MusicalGroup):
   equivalent_to = [
     onto.MusicalGroup
   & ( restriction("has_genre", SOME, onto.MusicalGenre)
     ) ]
   def eat(self): print("single genre group!")

NameError: ignored

In [ ]:
 from owlready2 import *

 onto = get_ontology("http://test.org/onto.owl")

 with onto:
     class Drug(Thing):
         def take(self): print("I took a drug")

     class ActivePrinciple(Thing):
         pass

     class has_for_active_principle(Drug >> ActivePrinciple):
         python_name = "active_principles"

     class Placebo(Drug):
         equivalent_to = [Drug & Not(has_for_active_principle.some(ActivePrinciple))]
         def take(self): print("I took a placebo")
     class SingleActivePrincipleDrug(Drug):
         equivalent_to = [Drug & has_for_active_principle.exactly(1, ActivePrinciple)]
         def take(self): print("I took a drug with a single active principle")

     class DrugAssociation(Drug):
         equivalent_to = [Drug & has_for_active_principle.min(2, ActivePrinciple)]
         def take(self): print("I took a drug with %s active principles" % len(self.active_principles))

acetaminophen   = ActivePrinciple("acetaminophen")
amoxicillin     = ActivePrinciple("amoxicillin")
clavulanic_acid = ActivePrinciple("clavulanic_acid")

AllDifferent([acetaminophen, amoxicillin, clavulanic_acid])

drug1 = Drug(active_principles = [acetaminophen])
drug2 = Drug(active_principles = [amoxicillin, clavulanic_acid])
drug3 = Drug(active_principles = [])
close_world(Drug)

# VoID


In [57]:
from rdflib import URIRef, Graph, Literal
from rdflib.namespace import VOID, RDF
from rdflib.void import generateVoID


graph = generateVoID(graph)